In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import time
import networkx as nx
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.models import Sequential
from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras_tqdm import TQDMNotebookCallback


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier

### Load data

In [13]:
    
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
PatchyConverter = GraphConverter(dataset_name, width, receptive_field)
PatchyConverter.relabel_nodes()
graph_tensor = PatchyConverter.graphs_to_Patchy_tensor()
# plt.imshow(mutag_tensor[0,:,:,2])

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)

# Assigning the data:
x_train, x_test, y_train, y_test = train_test_split(graph_tensor, graph_labels, test_size=0.10)
data = ((x_train,y_train),(x_test, y_test))
input_shape = x_train.shape[1:]

MUTAG tensor exists, loading it from Dropbox


In [14]:
x = PatchyConverter.get_average_num_nodes()
print(x)

17.930851063829788


In [ ]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))



class ConvNetPatchy(object):
    def __init__(self,graph_tensor, graph_labels):
        self.graph_tensor = graph_tensor
        self.graph_labels = graph_labels
        self.name = 'cnn'
        self.input_shape = graph_tensor.shape[1:]
        self.n_class = len(np.unique(graph_labels))
        
    def split_data(self,random_state = 42, test_size = 0.10):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.graph_tensor, self.graph_labels, test_size=test_size,random_state = random_state)
        self.y_train = pd.get_dummies(self.y_train).values
        self.y_test = pd.get_dummies(self.y_test).values
            
    def build_graph(self,):
        
        K.clear_session()
        self.model = Sequential()
        self.model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=self.input_shape))
        #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.model.add(Conv2D(8, kernel_size=(5, 5), activation='relu'))
        #model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.75))
        self.model.add(Dense(self.n_class, activation='softmax'))
        
    def train_model(self,epochs = 200, batch_size = 100,verbose = 0):
        self.epochs = epochs
        self.history = AccuracyHistory()
        self.model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(decay=1e-6),
              metrics=['accuracy'])
        
        self.model.fit(self.x_train, self.y_train,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=verbose,
                       validation_data=(self.x_test, self.y_test),
                       callbacks=[self.history])
        self.final_acc = self.history.acc[-1]        
        self.final_val_acc = self.history.val_acc[-1]


In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(mutag_tensor, mutag_labels, test_size=0.10)
# y_train = pd.get_dummies(y_train).values
# y_test = pd.get_dummies(y_test).values


# batch_size = 100
# num_classes= y_train.shape[1]
# epochs = 200
# input_shape = x_train.shape[1:]


In [ ]:
val_acc = []
for j in range(1):
    # Starting Conv net with Patchy
    patchy_cnn = ConvNetPatchy(graph_tensor, graph_labels)
    patchy_cnn.split_data(random_state = j)
    patchy_cnn.build_graph()
    patchy_cnn.train_model(epochs=400)
    print('Fold {} completed with val acc : {}'.format(j,patchy_cnn.final_val_acc))
    
    val_acc.append(patchy_cnn.final_val_acc)

In [ ]:
print('avg val accuracy: {}'.format(np.mean(val_acc)))
print('std val accuracy: {}'.format(np.std(val_acc)))
      

In [ ]:
score = patchy_cnn.model.evaluate(patchy_cnn.x_test, patchy_cnn.y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.acc)
plt.legend('train data',loc='lower center')
plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.val_acc)
plt.legend('test data')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()